In [1]:
import numpy as np
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.cross_validation import random_train_test_split
from scipy.sparse import coo_matrix as sp

C:\Users\Rodrigopareja\AppData\Local\Continuum\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
import pandas as pd
user_prod=pd.read_csv("user_prod.csv")
user_features=pd.read_csv("user_features.csv")
prod_features=pd.read_csv("prod_features.csv")

In [3]:
user_prod.head()

,hyb_dat_correo,id_producto,unidades_totales
0,fransches_11@hotmail.com,P0210060001,1
1,pattyhurtado29@gmail.com,P0292139009,6
2,marina_jose1997@hotmail.com,P0197053001,1
3,sandro.mas.2317@gmail.com,200088384,1
4,janita291279@gmail.com,P0261116008,3


In [4]:
prod_features.head()

,id_producto,nom_producto,desmarca,descategoria,dessubcategoriasolo,vta_neta_prom,Rango_Precios
0,200084474,CZ FASHION 101 50ML,CYZONE,FRAGANCIAS,DAMAS,9.93533,M10
1,P0197124004,CY ESM KHALI FIN SUGAR 5ML,CYZONE,MAQUILLAJE,PARA U#AS,3.07000,M5
2,200035116,ES AGU COL LAVANDA 1L,ESIKA,FRAGANCIAS,BEBES UNISEX,9.19160,M10
3,200098648,ES PERFECT BLOCK FACIAL 50ML,ESIKA,TRATAMIENTO FACIAL,PARA ROSTRO,6.72314,M10
4,200051088,LB ESSENTIAL DEMAQ 45ML,LBEL,TRATAMIENTO FACIAL,PARA ROSTRO,3.66227,M5


In [5]:
user_features.head()

,hyb_dat_correo,hyb_codpais,cosecha,CLV,TAG_SEGM,Recencia,Valor
0,deysi_aries01@hotmail.com,PE,2018-4,0.044609,R22,Mayor,B
1,eloaperez@belcorp.biz,PE,2018-1,0.150187,OUTL,Outl,B
2,evega@belcorp.biz,PE,2018-1,0.159832,R22,Mayor,B
3,angitalugo@gmail.com,PE,2018-1,0.168166,R22,Mayor,B
4,caranda@belcorp.biz,PE,2018-1,0.252150,R20,Mayor,B


In [6]:
prod_features[['vta_neta_prom']].describe()

,vta_neta_prom
count,2396.000000
mean,8.808033
std,5.138264
min,0.000000
25%,4.990613
50%,7.729875
75%,11.357500
max,32.985000


In [7]:
user_prod_interaction = pd.pivot_table(user_prod, index='hyb_dat_correo', columns='id_producto', values='unidades_totales')# fill missing values with 0
user_prod_interaction = user_prod_interaction.fillna(0)

In [8]:
user_prod_interaction

id_producto,200000441,200004616,200007423,200009812,200010080,200011767,200011773,200012014,200012707,200012761,...,P0292139009,P0292141004,P0292141009,P0297050000,P0297050001,P0297050002,P0297050004,P0297050008,P0297051000,P0454050000
hyb_dat_correo,,,,,,,,,,,,,,,,,,,,,
014200549a@uandina.edu.pe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
015200818a@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
017101636k@uandina.edu.pe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0412191086@uladech.pe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0420judith.carmen@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zurinna09@hotmail.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zusabino@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zuzziehg@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
user_id = list(user_prod_interaction.index)
user_dict = {}
counter = 0 
for i in user_id:
    user_dict[i] = counter
    counter += 1

In [10]:
# convert to csr matrix
from scipy.sparse import csr_matrix
user_prod_interaction_csr = csr_matrix(user_prod_interaction.values)
user_prod_interaction_csr

<24755x1988 sparse matrix of type '<class 'numpy.float64'>'
	with 101086 stored elements in Compressed Sparse Row format>

In [11]:
item_dict ={}
df = prod_features[['id_producto', 'nom_producto']].sort_values('id_producto').reset_index()
for i in range(df.shape[0]):
    item_dict[(df.loc[i,'id_producto'])] = df.loc[i,'nom_producto']
    

In [12]:
# dummify categorical features - product
prod_metadata_selected_transformed = pd.get_dummies(prod_features, columns = ['desmarca', 'descategoria', 'dessubcategoriasolo'])
prod_metadata_selected_transformed = prod_metadata_selected_transformed.sort_values('id_producto').reset_index().drop('index', axis=1)
prod_metadata_selected_transformed.head(10)



,id_producto,nom_producto,vta_neta_prom,Rango_Precios,desmarca_CYZONE,desmarca_ESIKA,desmarca_GENERICA,desmarca_LBEL,descategoria_ACCESORIOS COSMETICOS,descategoria_CABALLEROS,...,dessubcategoriasolo_TALCOS,dessubcategoriasolo_TEMATICOS,dessubcategoriasolo_TENDENCIA,dessubcategoriasolo_TINTES,dessubcategoriasolo_TIPO,dessubcategoriasolo_TRATAMIENTO CAPILAR,dessubcategoriasolo_TRATAMIENTO CORPORAL,dessubcategoriasolo_TRATAMIENTO FACIAL,dessubcategoriasolo_UNISEX,dessubcategoriasolo_VARIOS
0,200000441,ES AGU SH MANZANILLA 450ML,6.66375,M10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,200004616,LB DERMOCLARITE AM 30ML,16.33770,M20,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,200004616,LB DERMO CLARITE AM 30ML,13.00482,M15,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,200007423,LBEL BROCHA RUBOR,5.58090,M10,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,200009812,ES EXUS 100ML,12.70460,M15,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,200010080,ES FIORI 50ML,9.14290,M10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,200011767,ES IT'S YOU 50ML,7.08035,M10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,200011773,ES IT'S YOU 100ML,8.66885,M10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,200012014,ES KALOS SPORT 100ML,11.91980,M15,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,200012707,LB LIASSON 50ML,18.67769,M20,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# convert to csr matrix
prod_metadata_csr = csr_matrix(prod_metadata_selected_transformed.drop(['id_producto','nom_producto', 'vta_neta_prom','Rango_Precios'], axis=1).values)

In [14]:
prod_metadata_csr

<2396x64 sparse matrix of type '<class 'numpy.uint8'>'
	with 7188 stored elements in Compressed Sparse Row format>

In [15]:
# dummify categorical features - user
user_metadata_selected_transformed = pd.get_dummies(user_features, columns = ['Recencia','Valor'])
user_metadata_selected_transformed = user_metadata_selected_transformed.sort_values('hyb_dat_correo').reset_index().drop('index', axis=1)
user_metadata_selected_transformed.head(20)

,hyb_dat_correo,hyb_codpais,cosecha,CLV,TAG_SEGM,Recencia_Mayor,Recencia_Medio,Recencia_Menor,Recencia_Outl,Valor_A,Valor_B,Valor_M
0,014200549a@uandina.edu.pe,PE,2019-4,6.906293,R30,0,1,0,0,0,1,0
1,015200818a@gmail.com,PE,2020-1,16.499080,R11,0,0,1,0,0,0,1
2,017101636k@uandina.edu.pe,PE,2019-1,3.433145,R20,1,0,0,0,0,1,0
3,0412191086@uladech.pe,PE,2020-1,14.164280,R11,0,0,1,0,0,0,1
4,0420judith.carmen@gmail.com,PE,2018-4,2.925932,R20,1,0,0,0,0,1,0
5,04elianasilva@gmail.com,PE,2019-2,81.619632,R10,0,0,1,0,1,0,0
6,0510kslove@gmail.com,PE,2019-4,6.906293,R32,0,1,0,0,0,1,0
7,06lilidavila@gmail.com,PE,2020-1,17.332035,R11,0,0,1,0,0,0,1
8,08lilidavila@gmail.com,PE,2020-1,12.445452,R11,0,0,1,0,0,0,1
9,1000430300@certus.edu.pe,PE,2018-4,2.635428,R20,1,0,0,0,0,1,0


In [16]:
# convert to csr matrix
user_metadata_csr = csr_matrix(user_metadata_selected_transformed.drop(['hyb_dat_correo','hyb_codpais', 'CLV','cosecha','TAG_SEGM'], axis=1).values)

In [17]:
user_metadata_csr

<24766x7 sparse matrix of type '<class 'numpy.uint8'>'
	with 49532 stored elements in Compressed Sparse Row format>

In [18]:
model = LightFM(loss='warp',
                random_state=123,
                learning_rate=0.1,
                no_components=20,
                user_alpha=0.0)
model = model.fit(user_prod_interaction_csr,
                  epochs=10,
                  num_threads=1, verbose=False)

In [19]:
def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 10, show = True):
    
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items), item_features=prod_metadata_csr, 
                                     user_features=user_metadata_csr))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index).sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print ("Cliente: " + str(user_id))
        print("Ha comprado:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1
        print("\n Productos recomendados:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1

In [20]:
sample_recommendation_user(model, user_prod_interaction, '1063gap@gmail.com', user_dict, item_dict)

Cliente: 1063gap@gmail.com
Ha comprado:
1- ES ILUMINADOR LIQ RADIANCE
2- ES LABIAL HIDRACOLOR MATE
3- ES BASE HYDRA SMART
4- CZ SL LABIAL MATE 6ML

 Productos recomendados:
1- CY BOLSO KIM
2- CY BOLSO NICKY
3- ES PRO LAPIZ LABIAL BROCHA 1UN
4- FI CHARMING COIN EST ARE X4
5- LB DEVOS SPORT 100ML
6- ES IT'S YOU EMOTION 100ML
7- FI CHARMING COIN ARE
8- ES AGU SH REP MANZANILLA 750ML
9- ES VIBRANZA 45ML
10- ES PRO BROCHA POLVOS/RUBOR 1UN


In [ ]:
#Tabla de Output
#Puntos de automatización:
#1. Modelo
#2. API actualiza datos del modelo
#3. Job para volver a generar escenario
#4. Enviar resultado de SAP MKT a SAP Commerce
#llave: email 
#forma de carga
#productos a recomendar
#CUC a CodSAP